<a href="https://colab.research.google.com/github/hardiksahi/DeepLearning/blob/master/ConvertVOC2CSVFormat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code to convert PASCALVOC format (XML) to CSV format (path/to/image.jpg,x1,y1,x2,y2,class_name) 
https://github.com/ferrygun/PDFTableExtract/blob/master/build_logos.py

In [1]:
from bs4 import BeautifulSoup
import glob
from random import shuffle

In [2]:
import os
os.chdir("/content/drive/My Drive/Education/DL_ML/Colab_Notebooks/TableParser/")

In [3]:
print(os.getcwd())

/content/drive/My Drive/Education/DL_ML/Colab_Notebooks/TableParser


In [4]:
## Set path variables
#PDF_PATH = os.path.join(os.getcwd(), "PDF")
IMAGE_PATH = os.path.join(os.getcwd(), "Images")

In [7]:
#datasets = ['train', 'val', 'test']
datasets = ['test_new']

In [8]:
CLASSES = set()

for ds_type in datasets:
  output_csv = open(IMAGE_PATH+'/retinanet_'+ds_type+'.csv', "w")
  #print("output_csv path for {ds} is {p}".format(ds=ds_type, p=output_csv))

  all_images_path_list = [p for p in glob.glob(IMAGE_PATH+'/'+ds_type+'/*.jpg')] ## All the image paths in the folder.
  #print("All images in {ds} is {n}".format(ds=ds_type, n=len(all_images_path_list)))

  #tagged_images_path_list = [p for p in glob.glob(IMAGE_PATH+'/'+ds_type+'/labels_'+ds_type+'/*.xml')] ## All tagged images within that ds_type
  #print("Tagged images in {ds} is {n}".format(ds=ds_type, n=len(tagged_images_path_list)))

  shuffle(all_images_path_list)
  for image_path in all_images_path_list:
    image_name = (image_path.split('/')[-1]).split('.')[0] ## Image name without extension
    
    tagged_image_path = IMAGE_PATH+'/'+ds_type+'/labels_'+ds_type+'/'+image_name+'.xml'

    if os.path.isfile(tagged_image_path):
      contents = open(tagged_image_path).read()
      soup = BeautifulSoup(contents, "html.parser")
      coords = set()

      w = int(soup.find("width").string)
      h = int(soup.find("height").string)
      #print("Width:{w} and Height:{h}".format(w=w, h=h))

      for o in soup.find_all("object"):
        label = o.find("name").string
        xMin = int(o.find("xmin").string)
        yMin = int(o.find("ymin").string)
        xMax = int(o.find("xmax").string)
        yMax = int(o.find("ymax").string)

        xMin = max(0, xMin)
        yMin = max(0, yMin)
        xMax = min(w, xMax)
        yMax = min(h, yMax)

        coord = (xMin, yMin, xMax, yMax)

        if coord in coords:
          continue

        row = [image_path, str(xMin), str(yMin),str(xMax), str(yMax), label]
        output_csv.write("{}\n".format(",".join(row)))
        coords.add(coord)
        CLASSES.add(label)

    else:
      row = [image_path,'','','','','']
      output_csv.write("{}\n".format(",".join(row)))
  
  output_csv.close()    

In [7]:
write the classes to file
print("[INFO] writing classes...")
class_csv = open(IMAGE_PATH+'/retinanet_classes.csv', "w")
rows = [",".join([c, str(i)]) for (i, c) in enumerate(CLASSES)]
class_csv.write("\n".join(rows))
class_csv.close() 

[INFO] writing classes...


In [8]:
CLASSES

{'table'}